# 0. Install Dependencies

# 1. Test Random Environment with OpenAI Gym

In [1]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [19]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(5)
        # Temperature array
        self.observation_space = Discrete(100)
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
        self.answer = list(range(3))
        random.shuffle(self.answer)
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -2 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = 38 + random.randint(-3,3)
        # Reset shower time
        self.shower_length = 60
        random.shuffle(self.answer)
        return self.state

In [20]:
env = ShowerEnv()

In [4]:
env.observation_space.sample()

76

In [5]:
assert not env.observation_space.contains(1.5)

In [7]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:10
Episode:2 Score:-56
Episode:3 Score:-34
Episode:4 Score:-58
Episode:5 Score:-60
Episode:6 Score:-50
Episode:7 Score:-6
Episode:8 Score:-60
Episode:9 Score:4
Episode:10 Score:44


# 2. Create a Deep Learning Model with Keras

In [21]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [9]:
#states = env.observation_space.shape
states = (1,)
actions = env.action_space.n

In [10]:
actions

3

In [11]:
states

(1,)

In [13]:
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [62]:
del model

In [22]:
model = build_model(states, actions)

In [23]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 24)                48        
_________________________________________________________________
dense_7 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 75        
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


# 3. Build Agent with Keras-RL

In [17]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [18]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [24]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
    1/10000 [..............................] - ETA: 11:37 - reward: 1.0000

/Users/Ben/Home/Working/Hanabi/envs/lib/python3.7/site-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 71s 7ms/step - reward: -0.4972
166 episodes - episode_reward: -30.193 [-60.000, 60.000] - loss: 0.802 - mae: 7.679 - mean_q: -9.525

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 72s 7ms/step - reward: -0.0406
167 episodes - episode_reward: -2.072 [-60.000, 60.000] - loss: 1.381 - mae: 8.380 - mean_q: -5.159

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 74s 7ms/step - reward: 0.1818
167 episodes - episode_reward: 10.766 [-60.000, 60.000] - loss: 2.197 - mae: 8.740 - mean_q: 1.123

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 79s 8ms/step - reward: 0.3792
166 episodes - episode_reward: 22.602 [-60.000, 60.000] - loss: 6.342 - mae: 10.959 - mean_q: 12.510

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 83s 8ms/step - reward: 0.3358
done, took 378.766 seconds


In [26]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: -60.000, steps: 60
Episode 2: reward: 60.000, steps: 60
Episode 3: reward: 60.000, steps: 60
Episode 4: reward: -60.000, steps: 60
Episode 5: reward: 60.000, steps: 60
Episode 6: reward: -60.000, steps: 60
Episode 7: reward: -60.000, steps: 60
Episode 8: reward: -60.000, steps: 60
Episode 9: reward: -60.000, steps: 60
Episode 10: reward: -60.000, steps: 60
Episode 11: reward: 60.000, steps: 60
Episode 12: reward: 60.000, steps: 60
Episode 13: reward: 60.000, steps: 60
Episode 14: reward: -60.000, steps: 60
Episode 15: reward: 60.000, steps: 60
Episode 16: reward: 60.000, steps: 60
Episode 17: reward: 60.000, steps: 60
Episode 18: reward: -60.000, steps: 60
Episode 19: reward: -60.000, steps: 60
Episode 20: reward: 60.000, steps: 60
Episode 21: reward: -60.000, steps: 60
Episode 22: reward: -60.000, steps: 60
Episode 23: reward: -60.000, steps: 60
Episode 24: reward: 60.000, steps: 60
Episode 25: reward: -60.000, steps: 60
Episode 26: rewa

In [67]:
dqn.save_weights('dqn_weights_discrete.h5f', overwrite=True)

In [68]:
del model
del dqn
del env

In [69]:
env = ShowerEnv()
actions = env.action_space.n
states = (1,)
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [70]:
dqn.load_weights('dqn_weights_discrete.h5f')

In [71]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 58.000, steps: 60
Episode 2: reward: 58.000, steps: 60
Episode 3: reward: 58.000, steps: 60
Episode 4: reward: 60.000, steps: 60
Episode 5: reward: 60.000, steps: 60
Episode 6: reward: 58.000, steps: 60
Episode 7: reward: 58.000, steps: 60
Episode 8: reward: 60.000, steps: 60
Episode 9: reward: 60.000, steps: 60
Episode 10: reward: 58.000, steps: 60
Episode 11: reward: 58.000, steps: 60
Episode 12: reward: 60.000, steps: 60
Episode 13: reward: 60.000, steps: 60
Episode 14: reward: 60.000, steps: 60
Episode 15: reward: 58.000, steps: 60
Episode 16: reward: 60.000, steps: 60
Episode 17: reward: 60.000, steps: 60
Episode 18: reward: 60.000, steps: 60
Episode 19: reward: 60.000, steps: 60
Episode 20: reward: 60.000, steps: 60
Episode 21: reward: 60.000, steps: 60
Episode 22: reward: 60.000, steps: 60
Episode 23: reward: 60.000, steps: 60
Episode 24: reward: 60.000, steps: 60
Episode 25: reward: 60.000, steps: 60
Episode 26: reward: 60.000, st